In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import pandas as pd

In [ ]:
#네이버 웹툰 페이지 파싱을 통해 총 정식 연재 작품 수 구하기
URL='https://comic.naver.com/webtoon/weekday.nhn'
html=requests.get(URL).text
soup=BeautifulSoup(html,'html.parser')

title=soup.find_all('a',{'class': 'title'})
title_list=[] ; title_num=[]

#일주일 2회 이상 연재 작품은 한 번만 데이터 수집
for x in range(len(title)):
    t=title[x].text
    if(t in title_list):
        print(x, t)
        title_list.append(t)
        title_num.append(x)
    else:
        title_list.append(t)
        title_num.append(x)

print(title_num)
print(title_list)
# selenium으로 네이버 웹툰 페이지 불러오기 
URL='https://comic.naver.com/webtoon/weekday.nhn'
driver=webdriver.Chrome('chromedriver.exe')
driver.get(URL)

time.sleep(60)

artist_list=[] ; genre_list=[] ; score_list=[] ; id_list=[] ; nick_list=[] ; chat_list=[]
idlst=[] ; nicklst=[] ; chatlst=[] ; DataFrame_id={} ; DataFrame_chat={}

for i in title_num[245:]:
    
    id_list.clear()
    chat_list.clear()
    
    time.sleep(1)
    
    #전체 웹툰 목록 중 월요일 첫 번째 웹툰으로 페이지 이동
    page=driver.find_elements_by_class_name('title')#***********CLEAR v
    page[i].click()
    
    time.sleep(0.5)
    
    #이동한 페이지 주소 읽고, 파싱하기
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    #작가님 닉네임 수집
    artist = soup.find_all('h2')
    artist = artist[1].find('span',{'class':'wrt_nm'}).text[8:]
    artist_list.append(artist)
    
    #작품 장르 수집
    genre=soup.find('span',{'class':'genre'}).text
    genre_list.append([genre])
    
    #최신 별점 평균 점수 수집 (최대 10화 분량)
    score = soup.find_all('strong')
    scorelist=[] ; ii=9
    while score[ii].text[0].isnumeric()==True:
        scorelist.append(float(score[ii].text))
        ii +=1
    score_list.append(sum(scorelist)/len(scorelist))
    
    
    time.sleep(0.5)
    
    length=driver.find_elements_by_class_name('title')#*******CLEAR v
    if(len(length)>5):
        length=length[:2]
    
    #각 회차를 돌며 댓글 작성자 아이디 수집
    for j in range(len(length)):
        #해당 페이지의 회차 모두 가져오기
        titlenum=driver.find_elements_by_class_name('title')#*********CLEAR v
        
        time.sleep(0.5)
        
        webnum=[y.text for y in titlenum]
        enterToon = driver.find_elements_by_partial_link_text(webnum[j])#***CLEAR v
        
        time.sleep(0.5)
        
        enterToon[0].click()
        
        #현재 댓글 부분 파싱이 안되는 설정이므로 src의 댓글창 페이지로 직접 이동
        #현재 URL주소 복사
        html = driver.current_url
        currentPage=html[html.find('?'):]
        
        #댓글창 페이지 URL
        URL2='https://comic.naver.com/comment/comment.nhn' + currentPage
        print('Title:',title_list[i],i,' : ',URL2)
        
        time.sleep(0.4)
        
        #댓글창 페이지 열기
        driver2 = webdriver.Chrome('chromedriver.exe')
        driver2.get(URL2)
        
        time.sleep(3)
        
        try:
            #수집 편의를 위해 클린봇 댓글 필터링 기능 해제
            driver2.find_element_by_class_name('u_cbox_cleanbot_setbutton').click()
            time.sleep(0.4)
            driver2.find_element_by_class_name('u_cbox_layer_cleanbot_checkbox').click()
            time.sleep(0.4)
            driver2.find_element_by_class_name('u_cbox_layer_cleanbot_extrabutton').click()
            time.sleep(0.4)

            #베스트 댓글 아이디 목록 수집
            idbest = driver2.find_elements_by_class_name('u_cbox_id') ;jj=0 #*******CLEAR v

            time.sleep(0.1)

            nickbest = driver2.find_elements_by_class_name('u_cbox_nick') #*******CLEAR v

            time.sleep(0.4)

            clean=driver.find_elements_by_class_name('u_cbox_cleanbot_contents')
            if(len(clean)!=0):
                chat.append(clean[0].text)
                clean.clear()

                time.sleep(0.4)

            chatbest = driver2.find_elements_by_class_name('u_cbox_contents') #******CLEAR v

            time.sleep(0.4)

            while jj<len(idbest):
                id_list.append(idbest[jj].text)
                nick_list.append(nickbest[jj].text)
                chat_list.append(chatbest[jj].text)
                jj +=1

            time.sleep(0.3)

            #전체 댓글 보기 클릭 & 아이디 목록 수집
            driver2.find_element_by_partial_link_text('전체댓글').click()

            time.sleep(0.3)

            #최신 댓글부터 가장 나중의 댓글까지 창 이동하며 아이디 목록 수집
            num=0 ; start=0 ;
            while True:
                try:
                    while start<=0:

                        time.sleep(0.4)

                        page=driver2.find_elements_by_class_name('u_cbox_num_page')
                        print('now page: ',int(page[num].text),' index: ',num)

                        time.sleep(0.1)

                        idd=driver2.find_elements_by_class_name('u_cbox_id')

                        time.sleep(0.1)

                        nickk=driver2.find_elements_by_class_name('u_cbox_nick')

                        time.sleep(0.1)

                        chat=driver2.find_elements_by_class_name('u_cbox_contents')

                        time.sleep(0.1)

                        while len(idd)>0:
                            id_list.append(idd[0].text)
                            nick_list.append(nickk[0].text)
                            chat_list.append(chat[0].text)

                            idd.pop(0)
                            nickk.pop(0)
                            chat.pop(0)

                        time.sleep(0.1)

                        #페이지가 10의 배수이면 다음 페이지 클릭, 동일 과정 거침
                        try:
                            a=driver2.find_elements_by_class_name('u_cbox_next') #*****CLEAR v
                            if(int(page[num].text) % 10 == 0 and a[0].text != ''):
                                driver2.find_element_by_class_name('u_cbox_next').click()

                                time.sleep(0.1)

                                page=driver2.find_elements_by_class_name('u_cbox_num_page')

                                time.sleep(0.1)

                                idd=driver2.find_elements_by_class_name('u_cbox_id')

                                time.sleep(0.1)

                                nickk=driver2.find_elements_by_class_name('u_cbox_nick')

                                time.sleep(0.1)

                                chat=driver2.find_elements_by_class_name('u_cbox_contents')
                                a.clear()

                                time.sleep(0.1)

                                while len(idd)>0:
                                    id_list.append(idd[0].text)
                                    nick_list.append(nickk[0].text)
                                    chat_list.append(chat[0].text)

                                    idd.pop(0)
                                    nickk.pop(0)
                                    chat.pop(0)
                                num=0
                                print('now page: ',int(page[num].text),' index: ',num)

                        except IndexError:
                            break

                        num +=1

                        time.sleep(0.4)

                        try:
                            page[num].click()

                        except IndexError:
                            break

                        page.clear()



                        idd.clear()

                        time.sleep(0.1)

                    #댓글 페이지 닫기
                    driver2.close()
                    break

                except:
                    print('ERROR')
                    continue
        except:
            print('Non-exist')
            driver2.close()
            
            
            
        time.sleep(0.4)
        
        #페이지 뒤로 가기, 다시 만화 목록으로 
        driver.back()
        
        time.sleep(1)
        
        titlenum.clear()
        
        #time.sleep(0.5)
        
        enterToon.clear()
        
        #time.sleep(0.5)
        
        idbest.clear()
        nickbest.clear()
        chatbest.clear()
    
    #전체 정식연재 만화 창으로 이동
    time.sleep(1)
    print(title_list[i],"end")
    idlst.append(id_list.copy())
    nicklst.append(nick_list.copy())
    chatlst.append(chat_list.copy())
    
    driver.back()
    
    time.sleep(0.5)
    
    page.clear()
    
    time.sleep(0.5)
    length.clear()
    
#---------------------------------------------------크롤링 END--------------------------------------------------------------

#각 작품별로 데이터 개수가 가장 많은 작품만큼 '' (빈 값) 넣어줌
idc=idlst.copy()
idc.sort(key=lambda x : len(x),reverse=True)

for st in range(len(idc)):
    for stt in range(len(idc[st])):
        idlst[st][stt]=nicklst[st][stt]+idlst[st][stt]

for count in range(len(idc)):
    nn=len(idc[0])-len(chatlst[count])
    for plus in range(nn):
        idlst[count].append('')
        chatlst[count].append('')
    #별점 평균 점수, 장르, 작가명 넣어줌
for count1 in range(len(idc)):
    idlst[count1].insert(0,score_list[count1])
    idlst[count1].insert(0,genre_list[count1])
    idlst[count1].insert(0,artist_list[count1])
    chatlst[count1].insert(0,score_list[count1])
    chatlst[count1].insert(0,genre_list[count1])
    chatlst[count1].insert(0,artist_list[count1])

#전체 데이터를 데이터 프레임으로 변환
total_data_id={} ; total_data_chat={} ; d=0 
for diction in range(len(idc)):
    total_data_id[title_list[d]] = idlst[diction]
    total_data_chat[title_list[d]] = chatlst[diction]
    d +=1
    
Total_data_id = pd.DataFrame(total_data_id)
Total_data_chat = pd.DataFrame(total_data_chat)

#데이터 프레임을 CSV파일로 추출 내보내기(ID, CHAT 따로)
Total_data_id.to_csv('NaverWebtoon_id11', mode='w')
Total_data_chat.to_csv('NaverWebtoon_chat11', mode='w')